Example how to work with DBC database and prepare DBC channels mappings

In [7]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import pandas as pd
import canmatrix.formats
# from etils import ecolab
from pathlib import Path
from collections import defaultdict
import os
from utils.can_dbc_db import CanDbcDb
import  utils.can_logs as utls

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
dbc_dir=r'C:\Users\ivanm\Downloads\temp\FW\EKL_DP187_v0_20251020_180400_v0.7.0_5cb89b4\dbc'
# load databases
# dbc_db=DbcDb(dbc_dir,True,export_path=dbc_dir)
dbc_db=CanDbcDb(dbc_dir,True)

total number of id: 42
total number of dbc.msg: 57
number of dublicated id: 12
number of dublicated pgn: 0
possible id conflicts:
	one id but different message name
		id=32 msg names: {'HvchEcm_Lin1PartNr10Fr04', 'BFPV_StatusInfo'}
		id=4 msg names: {'XCU_BFPVCtrlCmd', 'EC_Status_3'}
		id=18 msg names: {'TCU_WP_EDS2_Control', 'VCU_WP_EDS2_Control'}
		id=17 msg names: {'WP_EDS2_Status', 'AC_Message'}
dbc db loaded


In [31]:
# how to get info
dbc_db.dbc_name_db['DTS_Status1.dbc'];
dbc_db.dbc_name_db['DTS_Status1.dbc']['DTS_Status1']['signals']['DTS_AccrPedlPosn'];
dbc_db.dbc_id_db[201];
dbc_db.dbc_pgn_db[65485];

defaultdict(<function utils.dbc_db.DbcDb.__init__.<locals>.<lambda>.<locals>.<lambda>()>,
            {'DTS_Status1': defaultdict(list,
                         {'info': {'id': 234488318,
                           'is_extended': True,
                           'is_j1939': True,
                           'comment': 'Message with general information about current Drivetrain System state',
                           'cycle_time': 50,
                           'attributes': {'GenMsgCycleTime': '50',
                            'GenMsgSendType': 'cyclic',
                            'VFrameFormat': 'J1939PG'},
                           'pgn': 129537,
                           'SA': 254,
                           'DA': None,
                           'dbc_dubl': ['BodyCan1.dbc::DTS_Status1',
                            'DTS_Status1.dbc::DTS_Status1']},
                          'signals': defaultdict(dict,
                                      {'DTS_Standstill': {'name': 'DTS_Standst

{'name': 'DTS_AccrPedlPosn',
 'unit': '%',
 'initial_value': 0.0,
 'min': 0.0,
 'max': 102.0,
 'factor': 0.4,
 'enumeration': None}

[{'dbc': 'ACU_Arrival.dbc',
  'msg': 'YRS_2',
  'is_j1939': False,
  'cycle_time': 10,
  'SendOnChange': None}]

[{'dbc': 'ADAS_FlashData.dbc',
  'msg': 'DCUFlashData',
  'is_j1939': True,
  'cycle_time': 0,
  'SendOnChange': None}]

In [10]:
# get dataframe and save it to the same dir
dbc_db.get_dbc_db_df(export_path=dbc_dir)
# dbc_db converted to dataframe and saved into df and  xlsx file
dbc_db.df.head(3);
#  df with signals
dbc_db.df_sig.head(3);

dbc_msg db saved to C:\Users\ivanm\Downloads\temp\FW\EKL_DP187_v0_20251020_180400_v0.7.0_5cb89b4\dbc\dbc_db.xlsx
dbc msg sig saved to C:\Users\ivanm\Downloads\temp\FW\EKL_DP187_v0_20251020_180400_v0.7.0_5cb89b4\dbc\dbcmsg_sig_db.xlsx


In [71]:
# how to prepare CAN map to be used in mdf.extract_bus_logging
DBC2CAN_MAPS_FILE=r'input\VAN_CAN_MAPS.xlsx'
can2ch_maps={'CAN2_VEH':2,'CAN7_BRK':7}
(CAN_MAP,can_db1)=utls.get_can_map2extract(dbc_dir,can2ch_maps,DBC2CAN_MAPS_FILE=r'input\VAN_CAN_MAPS.xlsx'
)
print(can_db1)
CAN_MAP.head()
# next use with extract_bus_logging
# mdf1_extracted = mdf1.extract_bus_logging(database_files=can_db1)


{'CAN': [(WindowsPath('dbc/ucs_dbc-0.12.0.0/DTS_Status1.dbc'), 2), (WindowsPath('dbc/ucs_dbc-0.12.0.0/GS.dbc'), 2), (WindowsPath('dbc/ucs_dbc-0.12.0.0/SimpVSM.dbc'), 2), (WindowsPath('dbc/ucs_dbc-0.12.0.0/ACU_Arrival.dbc'), 7), (WindowsPath('dbc/ucs_dbc-0.12.0.0/BRK_Status1.dbc'), 7), (WindowsPath('dbc/ucs_dbc-0.12.0.0/Chassis_CAN_v3_5_ModifiedInt.dbc'), 7), (WindowsPath('dbc/ucs_dbc-0.12.0.0/DTS_API.dbc'), 7), (WindowsPath('dbc/ucs_dbc-0.12.0.0/DTS_Status1.dbc'), 7), (WindowsPath('dbc/ucs_dbc-0.12.0.0/j1939.dbc'), 7), (WindowsPath('dbc/ucs_dbc-0.12.0.0/NTO_ARRIVAL_MANDO_CAN_SPEC_P.dbc'), 7), (WindowsPath('dbc/ucs_dbc-0.12.0.0/SC23S_general_customer.dbc'), 7)]}


c:\D\work\py_work\UCSDT_utils\can_log_mdf_parse\utils\can_log_utils.py:1072: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  CAN_MAP.replace({1: True, 0: False},inplace=True)


,dbc,CAN2_VEH,CAN7_BRK
1,ACU_Arrival.dbc,False,True
24,BRK_Status1.dbc,False,True
32,Chassis_CAN_v3_5_ModifiedInt.dbc,False,True
50,DTS_API.dbc,False,True
53,DTS_Status1.dbc,True,True
